In [3]:
#파이썬 버전 확인
!python --version


Python 3.8.5


In [4]:
!pip install --upgrade pip

In [5]:
#업그레이드

!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

!pip install konlpy

from konlpy.tag import Kkma

kkma = Kkma()

Processing c:\users\sm2141\untitled folder\jpype1-1.1.2-cp38-cp38-win_amd64.whl
JPype1 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [6]:
import pandas as pd

In [7]:
df_train = pd.read_csv('nsmc-master/ratings_train.txt', delimiter='\t')

In [8]:
df_train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [10]:
#결측치 제거 
df_train.dropna(inplace = True)

In [11]:
df_test = pd.read_csv('nsmc-master/ratings_test.txt', delimiter='\t')
df_test.dropna(inplace = True)

In [12]:
X_train = df_train['document']
y_train = df_train['label']
X_test = df_test['document']
y_test = df_test['label']
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train[:3])

TfidfVectorizer()

In [15]:
X_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [16]:
tfidf.vocabulary

In [17]:
kkma.nouns(X_train[0])

['더빙', '목소리']

In [18]:
kkma.morphs(X_train[0])

['아', '아', '더빙', '..', '진짜', '짜증나', '네요', '목소리']

In [19]:
kkma.pos(X_train[0])

[('아', 'VV'),
 ('아', 'ECS'),
 ('더빙', 'NNG'),
 ('..', 'SW'),
 ('진짜', 'MAG'),
 ('짜증나', 'VV'),
 ('네요', 'EFN'),
 ('목소리', 'NNG')]

In [21]:
d = pd.DataFrame(kkma.pos(X_train[0]), columns=['morphs','tag'])

In [22]:
d.set_index('tag', inplace = True)

In [23]:
d.loc[['VV','NNG']]

,morphs
tag,
VV,아
VV,짜증나
NNG,더빙
NNG,목소리


In [24]:
#tfidf의 토큰화 방법 변경


In [25]:
#명사 단위로 토큰화
def myTokenizer(text):
    return kkma.nouns(text)

In [26]:
#tfidf 내가 만든 토큰 방법 적용
tfidf = TfidfVectorizer(tokenizer = myTokenizer)

In [27]:
tfidf.fit(X_train[:3])
tfidf.vocabulary_

C:\Users\SM2141\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

In [28]:
#VV, VA, NNG의 형태소만 토큰화
def myTokenizer2(text):
    d = pd.DataFrame(kkma.pos(text), columns=['morph','tag'])
    d.set_index('tag', inplace=True)
    if ('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index):
        labels = ['VV','VA','NNG']
        return d.loc[d.index.intersection(labels)].dropna()['morph'].values
    else :
        return []

In [31]:
tfidf = TfidfVectorizer(tokenizer = myTokenizer2)
tfidf.fit(X_train[:3])
tfidf.vocabulary_

C:\Users\SM2141\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

In [33]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

In [36]:
pipe = make_pipeline(TfidfVectorizer(tokenizer = myTokenizer2) , LinearSVC())

In [37]:
pipe.fit(X_train[:100],y_train[:100])

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x0000022ECA96EB80>)),
                ('linearsvc', LinearSVC())])

In [38]:
#tfidf
pipe_tfidf = pipe.steps[0][1]

#svm
pipe_svm = pipe.steps[1][1]


In [40]:
voc = pd.DataFrame(pipe_tfidf.vocabulary_.keys(),
            index = pipe_tfidf.vocabulary_.values())

In [42]:
#Series로 확인
voc.sort_index()[0]

0         ㄱ
1         ㅇ
2         ㅈ
3         가
4        가볍
       ... 
495      활력
496    횡단보도
497       흠
498      흥행
499       희
Name: 0, Length: 500, dtype: object

In [43]:
#토큰들과 가중치를 연결해서 각각의 토큰들이 긍정.부정에 얼마나 영향을 끼쳤는지 확인
pipe_svm.coef_

array([[ 4.36018269e-01, -6.03071498e-01, -3.75530059e-01,
        -6.69215257e-02,  7.64272324e-02,  2.71457272e-01,
         4.73613190e-01, -1.57267645e-01, -1.04483843e-01,
        -4.04420007e-01, -1.57267645e-01, -6.93481535e-02,
        -2.30530174e-01,  2.04475732e-01,  4.73613190e-01,
         3.65070911e-01,  1.61069906e-01, -2.26962408e-01,
        -2.01023833e-01,  1.25191686e-01,  3.28339511e-01,
        -2.30375823e-01,  7.06694285e-02, -2.67125352e-01,
         2.33207920e-01, -2.72702235e-01, -3.68655798e-01,
        -1.57267645e-01, -1.34806669e-01, -3.02067545e-01,
         2.70644013e-01, -3.49122360e-01,  5.63367403e-01,
        -2.75556389e-01, -2.58284728e-01,  2.60282700e-01,
        -8.79509787e-01, -3.53319702e-01, -1.40947067e-01,
         2.05896222e-01,  3.64305119e-01, -3.45889304e-01,
         1.73521800e-01, -1.96895610e-01, -1.15827849e-01,
        -2.02483497e-01,  1.51388592e-01,  7.44301914e-02,
        -2.53816204e-02,  1.51388592e-01,  2.17880526e-0

In [45]:
pipe_svm.coef_.T

array([[ 4.36018269e-01],
       [-6.03071498e-01],
       [-3.75530059e-01],
       [-6.69215257e-02],
       [ 7.64272324e-02],
       [ 2.71457272e-01],
       [ 4.73613190e-01],
       [-1.57267645e-01],
       [-1.04483843e-01],
       [-4.04420007e-01],
       [-1.57267645e-01],
       [-6.93481535e-02],
       [-2.30530174e-01],
       [ 2.04475732e-01],
       [ 4.73613190e-01],
       [ 3.65070911e-01],
       [ 1.61069906e-01],
       [-2.26962408e-01],
       [-2.01023833e-01],
       [ 1.25191686e-01],
       [ 3.28339511e-01],
       [-2.30375823e-01],
       [ 7.06694285e-02],
       [-2.67125352e-01],
       [ 2.33207920e-01],
       [-2.72702235e-01],
       [-3.68655798e-01],
       [-1.57267645e-01],
       [-1.34806669e-01],
       [-3.02067545e-01],
       [ 2.70644013e-01],
       [-3.49122360e-01],
       [ 5.63367403e-01],
       [-2.75556389e-01],
       [-2.58284728e-01],
       [ 2.60282700e-01],
       [-8.79509787e-01],
       [-3.53319702e-01],
       [-1.4

In [47]:
result = pd.DataFrame(pipe_svm.coef_.T, index = voc.sort_index()[0],
            columns = ['w'])

In [49]:
result.sort_values(by = 'w', ascending = False)

,w
0,
재미있,0.928463
재밌,0.895683
인상적,0.761269
드라마,0.725361
내일,0.661585
...,...
추천,-0.673394
실망,-0.675704
졸작,-0.675713
